# Separador archivo de conciliación BECH3

In [23]:
import pandas as pd
import datetime
import os
from tkinter import filedialog

username = os.getlogin()

hoy = datetime.datetime.now()
date_format = "%d.%m.%Y"
date_formatexport = "%d-%m-%Y"
date_formatcartola = "%d/%m/%Y"
fecha_actual = hoy.strftime(date_format)
fecha_cartola = hoy.strftime(date_formatexport)

rows_per_file = 998

fp1 = filedialog.askopenfilename(initialdir='C:\\Users\\' + username + '\\Documents\\Py\\SEPARADOR',
                                 title="Ubicar archivo de conciliacion bancaria")
df1 = pd.read_table(fp1,
                  index_col=None,
                  dtype='object',
                  sep=';',
                  header=None,
                  names=['saldoa', 'diames', 'operacion', 'ind', 'sucursal', 'tipoabono', 'nsucursal', 'descripcion',
                         'monto', 'saldob', 'ind2', 'fecha', 'cuenta', 'blank1', 'blank2', 'blank3', 'blank4',
                         'blank5'],
                  )
df = df1.copy()

df = df[~df['tipoabono'].str.contains('C')]

df = df.drop(columns=['saldoa', 'diames', 'operacion', 'ind', 'sucursal', 'nsucursal', 'tipoabono',
                      'saldob', 'ind2', 'fecha', 'cuenta', 'blank1', 'blank2', 'blank3', 'blank4', 'blank5'])

num_files = -(-len(df) // rows_per_file)


Cargando archivos...
Archivos cargados


# Separar y exportar

In [7]:

for i in range(num_files):
    start = i * rows_per_file
    end = (i + 1) * rows_per_file

    chunk = df.iloc[start:end]

    sum_monto = chunk['monto'].astype(int).sum()

    chunk.loc[len(df)] = [None, str(int(sum_monto))]

    output_file = f'CargasBECH3_{i + 1}.csv'

    exportdir = 'C:\\Users\\' + username + '\\Documents\\Py\\SEPARADOR\\GENERADOS\\' + output_file + ''

    chunk.to_csv(exportdir, sep=';', index=False, header=False, float_format='%.0f')

    print(f'Se exportaron {len(chunk)} columnas al archivo {output_file}')


20


# Identificador cartola

Asigna un numero de agencia a cada monto en cartola según el monto de cierre con ajuste del archivo de ajustes histórico.

In [9]:
id_abonos = df1['abonos'].astype(str).str[-5:]
df2['mcca_id'] = df2['mcca'].astype(str).str.replace(r'\D+','',regex=True)
id_mcca = df2['mcca_id']

agencia_map = dict(zip(id_mcca,df2['agencia']))

df1['agencia'] = df1['abonos'].astype(str).apply(lambda x: agencia_map.get(x[-5:],0))

print(df1)

           fecha         sucursal  operacion  \
0     01/08/2024   Stgo.principal    7076406   
1     01/08/2024   Stgo.principal       1562   
2     01/08/2024   Stgo.principal       1562   
3     01/08/2024   Stgo.principal       1562   
4     01/08/2024   Stgo.principal       1562   
...          ...              ...        ...   
6013  31/07/2024   Stgo.principal       1562   
6014  31/07/2024   Stgo.principal       1562   
6015  31/07/2024   Stgo.principal       1562   
6016  31/07/2024   Stgo.principal    7014387   
6017  31/07/2024  Servi sn carlos          0   

                                            descripcion  cargos  abonos  \
0     Transferencia bancoestado de rut 13681745-0 fi...       0     560   
1                      Deposito en efectivo caja vecina       0  134450   
2                      Deposito en efectivo caja vecina       0  134220   
3                      Deposito en efectivo caja vecina       0  132790   
4                      Deposito en efectivo caja

Exportar

In [17]:
df1.to_csv('C:\\Users\\' + username + '\\Documents\\Py\\DEVOLTX\\Reporte CARGAS' + fecha_actual + '.csv',
            sep=';',
            header=False,
            index=False)

In [5]:
tipos_fecha = df1['fecha'].unique()
tipos_sucursal = df1['sucursal'].unique()
tipos_operacion = df1['operacion'].unique()
tipos_descripcion = df1['descripcion'].unique()